In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model

In [33]:
from sklearn.model_selection import train_test_split

In [20]:
import os
from keras import preprocessing
import keras

In [11]:
from keras.preprocessing.text import Tokenizer

In [1]:
import numpy as np
import pandas as pd

In [4]:
def load_data(df):
    question1 = df['question1'].astype(str).values
    question2 = df['question2'].astype(str).values
    # combined: to get the tokens
    df['combined'] = df['question1'] + df['question2']
    labels = df['is_duplicate'].values
    return question1, question2, labels

In [5]:
df = pd.read_csv('QQP.csv')
question1, question2, labels = load_data(df)
question1 = list(question1)
question2 = list(question2)
combined = question1 + question2
df.head()

,question1,question2,is_duplicate,combined
0,How is Pulsar 220?,Is Pulsar 220 good?,1,How is Pulsar 220?Is Pulsar 220 good?
1,How much marks one need to score in GATE to ge...,How much marks one need to score in GATE to ge...,0,How much marks one need to score in GATE to ge...
2,How does GoldFlake get the smoothest cigarette...,What are the best cigarettes you can get at 7-11?,0,How does GoldFlake get the smoothest cigarette...
3,How I increase my focus in study?,How do we increase concentration?,1,How I increase my focus in study?How do we inc...
4,I want to do biotechnology from abroad which c...,Why doesn’t Sterling Immigration update their ...,0,I want to do biotechnology from abroad which c...


In [7]:
def cleanAscii(text):
       return ''.join(i for i in text if ord(i) < 128)

In [8]:
for ind in df.index:
    df['combined'][ind] = cleanAscii(df['combined'][ind])

<ipython-input-8-8c200e623f49>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined'][ind] = cleanAscii(df['combined'][ind])


In [9]:
df.head()

,question1,question2,is_duplicate,combined
0,How is Pulsar 220?,Is Pulsar 220 good?,1,How is Pulsar 220?Is Pulsar 220 good?
1,How much marks one need to score in GATE to ge...,How much marks one need to score in GATE to ge...,0,How much marks one need to score in GATE to ge...
2,How does GoldFlake get the smoothest cigarette...,What are the best cigarettes you can get at 7-11?,0,How does GoldFlake get the smoothest cigarette...
3,How I increase my focus in study?,How do we increase concentration?,1,How I increase my focus in study?How do we inc...
4,I want to do biotechnology from abroad which c...,Why doesn’t Sterling Immigration update their ...,0,I want to do biotechnology from abroad which c...


In [12]:
max_words = 10000
tok = Tokenizer(num_words=max_words, oov_token="<OOV>")
tok.fit_on_texts(combined)

In [21]:
sequences = tok.texts_to_sequences(combined)
sequences = preprocessing.sequence.pad_sequences(sequences, maxlen=300, padding='post')

In [23]:
max_words = 10000
word_index = len(tok.word_index) + 1
glove_dir = ''
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'),encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))
print(word_index)
# matrix
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tok.word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

Found 400001 word vectors.
24384


In [27]:
lstm_units=50

In [28]:
lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2))
# loading our matrix
emb = tf.keras.layers.Embedding(max_words, embedding_dim, input_length=300, weights=[embedding_matrix],trainable=False)
input1 = tf.keras.Input(shape=(300,))
e1 = emb(input1)
x1 = lstm_layer(e1)
input2 = tf.keras.Input(shape=(300,))
e2 = emb(input2)
x2 = lstm_layer(e2)
mhd = lambda x: tf.keras.backend.abs(x[0] - x[1])
merged = tf.keras.layers.Lambda(function=mhd, output_shape=lambda x: x[0],name='L1_distance')([x1, x2])
preds = tf.keras.layers.Dense(1, activation='sigmoid')(merged)
model = tf.keras.Model(inputs=[input1, input2], outputs=preds)
model.compile(loss='mse', optimizer='adam')

In [34]:
def create_data():
    features, labels = df.drop(columns=['is_duplicate']).values, df['is_duplicate'].values
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)
    
    return x_train, x_test, y_train, y_test, x_val, y_val

In [35]:
x_train, x_test, y_train, y_test, x_val, y_val = create_data()

In [80]:
# history = model.fit([x_train[:,0], x_train[:,1]], y_train, epochs=100, validation_data=([x_val[:,0], x_val[:,1]], y_val), verbose=1)

In [51]:
EMBEDDING_DIM = 50

MAX_SEQUENCE_LENGTH = 10
VALIDATION_SPLIT = 0.1


RATE_DROP_LSTM = 0.17
RATE_DROP_DENSE = 0.25
NUMBER_LSTM = 50
NUMBER_DENSE_UNITS = 50
ACTIVATION_FUNCTION = 'relu'


siamese_config = {
	'EMBEDDING_DIM': EMBEDDING_DIM,
	'MAX_SEQUENCE_LENGTH' : MAX_SEQUENCE_LENGTH,
	'VALIDATION_SPLIT': VALIDATION_SPLIT,
	'RATE_DROP_LSTM': RATE_DROP_LSTM,
	'RATE_DROP_DENSE': RATE_DROP_DENSE,
	'NUMBER_LSTM': NUMBER_LSTM,
	'NUMBER_DENSE_UNITS': NUMBER_DENSE_UNITS,
	'ACTIVATION_FUNCTION': ACTIVATION_FUNCTION
}

In [54]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec
import numpy as np
import gc


def train_word2vec(documents, embedding_dim):
    model = Word2Vec(documents, min_count=1, vector_size=embedding_dim)
    word_vectors = model.wv
    del model
    return word_vectors


def create_embedding_matrix(tokenizer, word_vectors, embedding_dim):
    nb_words = len(tokenizer.word_index) + 1
    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((nb_words, embedding_dim))
    print("Embedding matrix shape: %s" % str(embedding_matrix.shape))
    for word, i in word_index.items():
        try:
            embedding_vector = word_vectors[word]
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
        except KeyError:
            print("vector not found for word - %s" % word)
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix


def word_embed_meta_data(documents, embedding_dim):
    documents = [x.lower().split() for x in documents]
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(documents)
    word_vector = train_word2vec(documents, embedding_dim)
    embedding_matrix = create_embedding_matrix(tokenizer, word_vector, embedding_dim)
    del word_vector
    gc.collect()
    return tokenizer, embedding_matrix


def create_train_dev_set(tokenizer, sentences_pair, is_similar, max_sequence_length, validation_split_ratio):eaks_val (np.array):
    sentences1 = [x[0].lower() for x in sentences_pair]
    sentences2 = [x[1].lower() for x in sentences_pair]
    train_sequences_1 = tokenizer.texts_to_sequences(sentences1)
    train_sequences_2 = tokenizer.texts_to_sequences(sentences2)
    leaks = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
             for x1, x2 in zip(train_sequences_1, train_sequences_2)]

    train_padded_data_1 = pad_sequences(train_sequences_1, maxlen=max_sequence_length)
    train_padded_data_2 = pad_sequences(train_sequences_2, maxlen=max_sequence_length)
    train_labels = np.array(is_similar)
    leaks = np.array(leaks)

    shuffle_indices = np.random.permutation(np.arange(len(train_labels)))
    train_data_1_shuffled = train_padded_data_1[shuffle_indices]
    train_data_2_shuffled = train_padded_data_2[shuffle_indices]
    train_labels_shuffled = train_labels[shuffle_indices]
    leaks_shuffled = leaks[shuffle_indices]

    dev_idx = max(1, int(len(train_labels_shuffled) * validation_split_ratio))

    del train_padded_data_1
    del train_padded_data_2
    gc.collect()

    train_data_1, val_data_1 = train_data_1_shuffled[:-dev_idx], train_data_1_shuffled[-dev_idx:]
    train_data_2, val_data_2 = train_data_2_shuffled[:-dev_idx], train_data_2_shuffled[-dev_idx:]
    labels_train, labels_val = train_labels_shuffled[:-dev_idx], train_labels_shuffled[-dev_idx:]
    leaks_train, leaks_val = leaks_shuffled[:-dev_idx], leaks_shuffled[-dev_idx:]

    return train_data_1, train_data_2, labels_train, leaks_train, val_data_1, val_data_2, labels_val, leaks_val


def create_test_data(tokenizer, test_sentences_pair, max_sequence_length):
    test_sentences1 = [x[0].lower() for x in test_sentences_pair]
    test_sentences2 = [x[1].lower() for x in test_sentences_pair]

    test_sequences_1 = tokenizer.texts_to_sequences(test_sentences1)
    test_sequences_2 = tokenizer.texts_to_sequences(test_sentences2)
    leaks_test = [[len(set(x1)), len(set(x2)), len(set(x1).intersection(x2))]
                  for x1, x2 in zip(test_sequences_1, test_sequences_2)]

    leaks_test = np.array(leaks_test)
    test_data_1 = pad_sequences(test_sequences_1, maxlen=max_sequence_length)
    test_data_2 = pad_sequences(test_sequences_2, maxlen=max_sequence_length)

    return test_data_1, test_data_2, leaks_test

In [55]:
# keras imports
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.models import Model

# std imports
import time
import gc
import os




class SiameseBiLSTM:
    def __init__(self, embedding_dim, max_sequence_length, number_lstm, number_dense, rate_drop_lstm, 
                 rate_drop_dense, hidden_activation, validation_split_ratio):
        self.embedding_dim = embedding_dim
        self.max_sequence_length = max_sequence_length
        self.number_lstm_units = number_lstm
        self.rate_drop_lstm = rate_drop_lstm
        self.number_dense_units = number_dense
        self.activation_function = hidden_activation
        self.rate_drop_dense = rate_drop_dense
        self.validation_split_ratio = validation_split_ratio

    def train_model(self, sentences_pair, is_similar, embedding_meta_data, model_save_directory='./'):urn (best_model_path):
        tokenizer, embedding_matrix = embedding_meta_data['tokenizer'], embedding_meta_data['embedding_matrix']

        train_data_x1, train_data_x2, train_labels, leaks_train, \
        val_data_x1, val_data_x2, val_labels, leaks_val = create_train_dev_set(tokenizer, sentences_pair,
                                                                               is_similar, self.max_sequence_length,
                                                                               self.validation_split_ratio)

        if train_data_x1 is None:
            print("++++ !! Failure: Unable to train model ++++")
            return None

        nb_words = len(tokenizer.word_index) + 1

        # Creating word embedding layer
        embedding_layer = Embedding(nb_words, self.embedding_dim, weights=[embedding_matrix],
                                    input_length=self.max_sequence_length, trainable=False)

        # Creating LSTM Encoder
        lstm_layer = Bidirectional(LSTM(self.number_lstm_units, dropout=self.rate_drop_lstm, recurrent_dropout=self.rate_drop_lstm))

        # Creating LSTM Encoder layer for First Sentence
        sequence_1_input = Input(shape=(self.max_sequence_length,), dtype='int32')
        embedded_sequences_1 = embedding_layer(sequence_1_input)
        x1 = lstm_layer(embedded_sequences_1)

        # Creating LSTM Encoder layer for Second Sentence
        sequence_2_input = Input(shape=(self.max_sequence_length,), dtype='int32')
        embedded_sequences_2 = embedding_layer(sequence_2_input)
        x2 = lstm_layer(embedded_sequences_2)

        # Creating leaks input
        leaks_input = Input(shape=(leaks_train.shape[1],))
        leaks_dense = Dense(int(self.number_dense_units/2), activation=self.activation_function)(leaks_input)

        # Merging two LSTM encodes vectors from sentences to
        # pass it to dense layer applying dropout and batch normalisation
        merged = concatenate([x1, x2, leaks_dense])
        merged = BatchNormalization()(merged)
        merged = Dropout(self.rate_drop_dense)(merged)
        merged = Dense(self.number_dense_units, activation=self.activation_function)(merged)
        merged = BatchNormalization()(merged)
        merged = Dropout(self.rate_drop_dense)(merged)
        preds = Dense(1, activation='sigmoid')(merged)

        model = Model(inputs=[sequence_1_input, sequence_2_input, leaks_input], outputs=preds)
        model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=3)

        STAMP = 'lstm_%d_%d_%.2f_%.2f' % (self.number_lstm_units, self.number_dense_units, self.rate_drop_lstm, self.rate_drop_dense)

        checkpoint_dir = model_save_directory + 'checkpoints/' + str(int(time.time())) + '/'

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        bst_model_path = checkpoint_dir + STAMP + '.h5'

        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)

        tensorboard = TensorBoard(log_dir=checkpoint_dir + "logs/{}".format(time.time()))

        model.fit([train_data_x1, train_data_x2, leaks_train], train_labels,
                  validation_data=([val_data_x1, val_data_x2, leaks_val], val_labels),
                  epochs=200, batch_size=64, shuffle=True,
                  callbacks=[early_stopping, model_checkpoint, tensorboard])

        return bst_model_path


    def update_model(self, saved_model_path, new_sentences_pair, is_similar, embedding_meta_data):
        tokenizer = embedding_meta_data['tokenizer']
        train_data_x1, train_data_x2, train_labels, leaks_train, \
        val_data_x1, val_data_x2, val_labels, leaks_val = create_train_dev_set(tokenizer, new_sentences_pair,
                                                                               is_similar, self.max_sequence_length,
                                                                               self.validation_split_ratio)
        model = load_model(saved_model_path)
        model_file_name = saved_model_path.split('/')[-1]
        new_model_checkpoint_path  = saved_model_path.split('/')[:-2] + str(int(time.time())) + '/' 

        new_model_path = new_model_checkpoint_path + model_file_name
        model_checkpoint = ModelCheckpoint(new_model_checkpoint_path + model_file_name,
                                           save_best_only=True, save_weights_only=False)

        early_stopping = EarlyStopping(monitor='val_loss', patience=3)

        tensorboard = TensorBoard(log_dir=new_model_checkpoint_path + "logs/{}".format(time.time()))

        model.fit([train_data_x1, train_data_x2, leaks_train], train_labels,
                  validation_data=([val_data_x1, val_data_x2, leaks_val], val_labels),
                  epochs=50, batch_size=3, shuffle=True,
                  callbacks=[early_stopping, model_checkpoint, tensorboard])

        return new_model_path

In [78]:
from operator import itemgetter
from keras.models import load_model
import pandas as pd

########################################
############ Data Preperation ##########
########################################


df = pd.read_csv('QQP.csv')

sentences1 = list(df['question1'])
sentences2 = list(df['question2'])
is_similar = list(df['is_duplicate'])
del df


####################################
######## Word Embedding ############
####################################


# creating word embedding meta data for word embedding 
tokenizer, embedding_matrix = word_embed_meta_data(sentences1 + sentences2,  siamese_config['EMBEDDING_DIM'])

embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}

## creating sentence pairs
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]
del sentences1
del sentences2


##########################
######## Training ########
##########################



class Configuration(object):
    """Dump stuff here"""

CONFIG = Configuration()
CONFIG.embedding_dim = siamese_config['EMBEDDING_DIM']
CONFIG.max_sequence_length = siamese_config['MAX_SEQUENCE_LENGTH']
CONFIG.number_lstm_units = siamese_config['NUMBER_LSTM']
CONFIG.rate_drop_lstm = siamese_config['RATE_DROP_LSTM']
CONFIG.number_dense_units = siamese_config['NUMBER_DENSE_UNITS']
CONFIG.activation_function = siamese_config['ACTIVATION_FUNCTION']
CONFIG.rate_drop_dense = siamese_config['RATE_DROP_DENSE']
CONFIG.validation_split_ratio = siamese_config['VALIDATION_SPLIT']

siamese = SiameseBiLSTM(CONFIG.embedding_dim , CONFIG.max_sequence_length, CONFIG.number_lstm_units , CONFIG.number_dense_units, 
					    CONFIG.rate_drop_lstm, CONFIG.rate_drop_dense, CONFIG.activation_function, CONFIG.validation_split_ratio)

best_model_path = siamese.train_model(sentences_pair, is_similar, embedding_meta_data, model_save_directory='./')


########################
###### Testing #########
########################

# model = load_model(best_model_path)

# test_sentence_pairs = [('What can make Physics easy to learn?','How can you make physics easy to learn?'),
# 					   ('How many times a day do a clocks hands overlap?','What does it mean that every time I look at the clock the numbers are the same?')]

# test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentence_pairs,  siamese_config['MAX_SEQUENCE_LENGTH'])

# preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
# results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
# results.sort(key=itemgetter(2), reverse=True)
# print(results)

Embedding matrix shape: (39989, 50)
Null word embeddings: 1
Epoch 1/200
330/330 [==============================] - 23s 25ms/step - loss: 0.6356 - acc: 0.6481 - val_loss: 0.5279 - val_acc: 0.7200
Epoch 2/200
330/330 [==============================] - 7s 22ms/step - loss: 0.5360 - acc: 0.7046 - val_loss: 0.5036 - val_acc: 0.7222
Epoch 3/200
330/330 [==============================] - 7s 21ms/step - loss: 0.5154 - acc: 0.7219 - val_loss: 0.4918 - val_acc: 0.7371
Epoch 4/200
330/330 [==============================] - 7s 22ms/step - loss: 0.5041 - acc: 0.7240 - val_loss: 0.4862 - val_acc: 0.7443
Epoch 5/200
330/330 [==============================] - 7s 23ms/step - loss: 0.4977 - acc: 0.7319 - val_loss: 0.4837 - val_acc: 0.7375
Epoch 6/200
330/330 [==============================] - 7s 21ms/step - loss: 0.4917 - acc: 0.7364 - val_loss: 0.4772 - val_acc: 0.7473
Epoch 7/200
330/330 [==============================] - 7s 21ms/step - loss: 0.4884 - acc: 0.7367 - val_loss: 0.4821 - val_acc: 0.7405
E

In [79]:
mqp = pd.read_csv('mqp.csv')

In [70]:
mqp.head()

,question1,question2,is_duplicate
0,After how many hour from drinking an antibioti...,I vomited this morning and I am not sure if it...,0
1,Am I over weight (192.9) for my age (39)?,I am a 39 y/o male currently weighing about 19...,1
2,Am I over weight (192.9) for my age (39)?,What diet is good for losing weight? Keto or v...,0
3,Aspirin allergy - is it worth getting a bracelet?,How much Aspirin can I take for my headache wi...,0
4,Aspirin allergy - is it worth getting a bracelet?,My friend told me about this bracelet for Aspi...,1


In [60]:
mqp = mqp.iloc[: , 1:]

In [61]:
mqp.columns = ['question1','question2','is_duplicate']

In [62]:
mqp.head()

,question1,question2,is_duplicate
0,After how many hour from drinking an antibioti...,I vomited this morning and I am not sure if it...,0
1,Am I over weight (192.9) for my age (39)?,I am a 39 y/o male currently weighing about 19...,1
2,Am I over weight (192.9) for my age (39)?,What diet is good for losing weight? Keto or v...,0
3,Aspirin allergy - is it worth getting a bracelet?,How much Aspirin can I take for my headache wi...,0
4,Aspirin allergy - is it worth getting a bracelet?,My friend told me about this bracelet for Aspi...,1


In [73]:
mqp.to_csv('mqp.csv', index=False)

In [69]:
mqp.head()

,question1,question2,is_duplicate
0,After how many hour from drinking an antibioti...,I vomited this morning and I am not sure if it...,0
1,Am I over weight (192.9) for my age (39)?,I am a 39 y/o male currently weighing about 19...,1
2,Am I over weight (192.9) for my age (39)?,What diet is good for losing weight? Keto or v...,0
3,Aspirin allergy - is it worth getting a bracelet?,How much Aspirin can I take for my headache wi...,0
4,Aspirin allergy - is it worth getting a bracelet?,My friend told me about this bracelet for Aspi...,1


In [74]:
m = pd.read_csv('mqp.csv')

In [75]:
m.head()

,question1,question2,is_duplicate
0,After how many hour from drinking an antibioti...,I vomited this morning and I am not sure if it...,0
1,Am I over weight (192.9) for my age (39)?,I am a 39 y/o male currently weighing about 19...,1
2,Am I over weight (192.9) for my age (39)?,What diet is good for losing weight? Keto or v...,0
3,Aspirin allergy - is it worth getting a bracelet?,How much Aspirin can I take for my headache wi...,0
4,Aspirin allergy - is it worth getting a bracelet?,My friend told me about this bracelet for Aspi...,1


In [ ]:
m = m.iloc